Imports

In [20]:
from os import getenv
import json

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sqlalchemy as db
from dotenv import load_dotenv

load_dotenv()

True

Connect to Database

In [3]:
# DB connection
cnx = db.create_engine(f"postgresql://{getenv('USER')}:{getenv('PASS')}@{getenv('HOST')}:{getenv('PORT')}/{getenv('DB')}")

# Elo Tests

In [49]:
# Auxiliary functions
def expected_score(rating_a, rating_b):
    expected_a = 1 / (1 + 10 ** ((rating_b - rating_a) / 400))
    expected_b = 1 / (1 + 10 ** ((rating_a - rating_b) / 400))
    return expected_a, expected_b

def K_days_update(K_lower, K_upper, players, game_date):
    D = sum([min(90, (game_date - players[item]).days) for item in players.keys()])
    return K_upper - (K_upper - K_lower) * D / 270


In [6]:
# Extract game history
with open('queries/elo_pre_query.sql') as f:
    sql = f.read()

df = pd.read_sql(sql, cnx, parse_dates=['date'])
df

,date,winner_team_id,winner_team_players,loser_team_id,loser_team_players
0,2015-09-08 00:00:00,6020bc70f1e4807cc700239e,"[5f3d8fdd95f40596eae23d7d, 5f3d8fdd95f40596eae...",6020be74f1e4807cc7013fcb,"[5f3d8fdd95f40596eae23f28, 5f3d8fdd95f40596eae..."
1,2015-09-08 00:00:00,6020bc70f1e4807cc700239e,"[5f3d8fdd95f40596eae23eb0, 5f3d8fdd95f40596eae...",6020be74f1e4807cc7013fcb,"[5f3d8fdd95f40596eae23f29, 5f3d8fdd95f40596eae..."
2,2015-09-08 00:00:00,6020bc70f1e4807cc700239e,"[5f3d8fdd95f40596eae23d7a, 5f3d8fdd95f40596eae...",6020be74f1e4807cc7013fcb,"[5f3d8fdd95f40596eae23f29, 5f3d8fdd95f40596eae..."
3,2015-09-08 00:00:00,6020be74f1e4807cc7013f9b,"[5f3d8fdd95f40596eae23da8, 5f3d8fdd95f40596eae...",6020be74f1e4807cc7014004,"[5f3d8fdd95f40596eae23f2e, 5f3d8fdd95f40596eae..."
4,2015-09-08 00:00:00,6020bc70f1e4807cc70023c7,"[5f3d8fdd95f40596eae23d97, 5f3d8fdd95f40596eae...",6020be74f1e4807cc7013fe8,"[5f3d8fdd95f40596eae23f2c, 5f3d8fdd95f40596eae..."
...,...,...,...,...,...
95859,2022-11-27 20:45:29,637b7fbcf73a2c40baedee3e,"[6101ba2e87f814e9fbffeb2e, 5f3d8fdd95f40596eae...",6378c7e85a20c5676abc7421,"[614ef898143c37878b237c9d, 6183c598f8090ec7452..."
95860,2022-11-27 20:53:10,6378c7e85a20c5676abc7421,"[6183c598f8090ec74528abd7, 5f3d8fdd95f40596eae...",637b7fbcf73a2c40baedee3e,"[63847e03f73a2c40baedf22b, 5f3d8fdd95f40596eae..."
95861,2022-11-27 21:04:11,6378c7e85a20c5676abc7421,"[5f3d8fdd95f40596eae23e0d, 614ef898143c37878b2...",637b7fbcf73a2c40baedee3e,"[63847e03f73a2c40baedf22b, 6101ba2e87f814e9fbf..."
95862,2022-11-27 21:14:06,637b7fbcf73a2c40baedee3e,"[63847e03f73a2c40baedf22b, 6101ba2e87f814e9fbf...",6378c7e85a20c5676abc7421,"[6183c598f8090ec74528abd7, 614ef898143c37878b2..."


In [7]:
# Extract team names
sql = 'select id, name from rocket_league.teams'

teams = pd.read_sql(sql, cnx)
teams

,id,name
0,6020bc70f1e4807cc7002386,Rogue
1,6020bc70f1e4807cc7002387,Lights Out!
2,6020bc70f1e4807cc7002389,Spacestation Gaming
3,6020bc70f1e4807cc700239e,Kings of Urban
4,6020bc70f1e4807cc700239d,Cloud9
...,...,...
4322,637b3d525a20c5676abd65e4,Ottawa University
4323,637b3d5f5a20c5676abd65e5,Indian River State College
4324,637b434a5a20c5676abd94ae,Mosquitoes
4325,637b7fbcf73a2c40baedee3e,Roehampton Esports Gold


## Naive Elo

In [39]:
current_ratings = {}
ratings_history = []

for i in range(len(df)):
    game = df.iloc[i,:]

    # Initialize entry if not exists
    if game['winner_team_id'] not in set(current_ratings.keys()):
        current_ratings[game['winner_team_id']] = {
            'date': game['date'],
            'rating': 2000,
        }
    
    if game['loser_team_id'] not in set(current_ratings.keys()):
        current_ratings[game['loser_team_id']] = {
            'date': game['date'],
            'rating': 2000,
        }

    # Update ratings
    k = 32

    rating_winner = current_ratings[game['winner_team_id']]['rating']
    rating_loser = current_ratings[game['loser_team_id']]['rating']

    expected_winner, expected_loser = expected_score(rating_winner, rating_loser)

    current_ratings[game['winner_team_id']]['rating'] = rating_winner + k * (1 - expected_winner)
    current_ratings[game['loser_team_id']]['rating'] = rating_loser + k * (0 - expected_loser)

    ratings_history.append({
        'team': game['winner_team_id'],
        'date': game['date'],
        'rating': current_ratings[game['winner_team_id']]['rating']
    })
    ratings_history.append({
        'team': game['loser_team_id'],
        'date': game['date'],
        'rating': current_ratings[game['loser_team_id']]['rating']
    })

ratings_history = pd.DataFrame(ratings_history)
ratings_history = ratings_history.merge(teams, left_on='team', right_on='id', how='left')
ratings_history.drop(columns=['id'], inplace=True)
ratings_history

,team,date,rating,name
0,6020bc70f1e4807cc700239e,2015-09-08 00:00:00,2016.000000,Kings of Urban
1,6020be74f1e4807cc7013fcb,2015-09-08 00:00:00,1984.000000,DeVoid Gaming
2,6020bc70f1e4807cc700239e,2015-09-08 00:00:00,2030.530498,Kings of Urban
3,6020be74f1e4807cc7013fcb,2015-09-08 00:00:00,1969.469502,DeVoid Gaming
4,6020bc70f1e4807cc700239e,2015-09-08 00:00:00,2043.747134,Kings of Urban
...,...,...,...,...
191723,637b7fbcf73a2c40baedee3e,2022-11-27 21:04:11,2010.795996,Roehampton Esports Gold
191724,637b7fbcf73a2c40baedee3e,2022-11-27 21:14:06,2028.580346,Roehampton Esports Gold
191725,6378c7e85a20c5676abc7421,2022-11-27 21:14:06,2031.920168,Dr. Buhmann Akademie H
191726,6378c7e85a20c5676abc7421,2022-11-27 21:21:34,2047.766368,Dr. Buhmann Akademie H


In [44]:
x = [current_ratings[item]['rating'] for item in current_ratings.keys()]
fig = go.Figure(data=[go.Histogram(x=x)])
fig.update_layout(title='Naive Elo Distribution')
fig

In [40]:
teams2plot = {
    'Spacestation Gaming', 'NRG Esports', 'Version1','FaZe Clan', 'FURIA Esports', 'Shopify Rebellion', 'Team AXLE', 'Gen.G Esports', 'G2 Esports'
}
plot_df = ratings_history[(ratings_history['name'].apply(lambda x: x in teams2plot)) & (ratings_history['date'].dt.year >= 2021)].copy()
plot_df['month'] = plot_df['date'].to_numpy().astype('datetime64[M]')
plot_df = plot_df.groupby(['month', 'name'], as_index=False).mean(['rating'])
px.line(plot_df, x='month', y='rating', color='name')

## Updating based on days
This first experiment is to change the update parameter $K$ based on the number of days each player has been on the team

In [50]:
current_ratings = {}
ratings_history = []
k_lower = 16
k_upper = 32

for i in range(len(df)):
    game = df.iloc[i,:]

    # Initialize entry if not exists
    if game['winner_team_id'] not in set(current_ratings.keys()):
        current_ratings[game['winner_team_id']] = {
            'date': game['date'],
            'rating': 2000,
            'players': {item: game['date'] for item in game['winner_team_players']}
        }
    
    if game['loser_team_id'] not in set(current_ratings.keys()):
        current_ratings[game['loser_team_id']] = {
            'date': game['date'],
            'rating': 2000,
            'players': {item: game['date'] for item in game['loser_team_players']}
        }

    # Update parameters
    winner_previous_players = set(current_ratings[game['winner_team_id']]['players'].keys())
    winner_current_players = set(game['winner_team_players'])
    for player in list(winner_previous_players - winner_current_players):
        current_ratings[game['winner_team_id']]['players'].pop(player)
    for player in list(winner_current_players - winner_previous_players):
        current_ratings[game['winner_team_id']]['players'][player] = game['date']

    loser_previous_players = set(current_ratings[game['loser_team_id']]['players'].keys())
    loser_current_players = set(game['loser_team_players'])
    for player in list(loser_previous_players - loser_current_players):
        current_ratings[game['loser_team_id']]['players'].pop(player)
    for player in list(loser_current_players - loser_previous_players):
        current_ratings[game['loser_team_id']]['players'][player] = game['date']

    kw = K_days_update(k_lower, k_upper, current_ratings[game['winner_team_id']]['players'], game['date'])
    kl = K_days_update(k_lower, k_upper, current_ratings[game['loser_team_id']]['players'], game['date'])

    # Update ratings
    rating_winner = current_ratings[game['winner_team_id']]['rating']
    rating_loser = current_ratings[game['loser_team_id']]['rating']

    expected_winner, expected_loser = expected_score(rating_winner, rating_loser)

    current_ratings[game['winner_team_id']]['rating'] = rating_winner + kw * (1 - expected_winner)
    current_ratings[game['loser_team_id']]['rating'] = rating_loser + kl * (0 - expected_loser)

    ratings_history.append({
        'team': game['winner_team_id'],
        'date': game['date'],
        'rating': current_ratings[game['winner_team_id']]['rating']
    })
    ratings_history.append({
        'team': game['loser_team_id'],
        'date': game['date'],
        'rating': current_ratings[game['loser_team_id']]['rating']
    })

ratings_history = pd.DataFrame(ratings_history)
ratings_history = ratings_history.merge(teams, left_on='team', right_on='id', how='left')
ratings_history.drop(columns=['id'], inplace=True)
ratings_history

,team,date,rating,name
0,6020bc70f1e4807cc700239e,2015-09-08 00:00:00,2016.000000,Kings of Urban
1,6020be74f1e4807cc7013fcb,2015-09-08 00:00:00,1984.000000,DeVoid Gaming
2,6020bc70f1e4807cc700239e,2015-09-08 00:00:00,2030.530498,Kings of Urban
3,6020be74f1e4807cc7013fcb,2015-09-08 00:00:00,1969.469502,DeVoid Gaming
4,6020bc70f1e4807cc700239e,2015-09-08 00:00:00,2043.747134,Kings of Urban
...,...,...,...,...
191723,637b7fbcf73a2c40baedee3e,2022-11-27 21:04:11,2008.969055,Roehampton Esports Gold
191724,637b7fbcf73a2c40baedee3e,2022-11-27 21:14:06,2026.530274,Roehampton Esports Gold
191725,6378c7e85a20c5676abc7421,2022-11-27 21:14:06,2027.569662,Dr. Buhmann Akademie H
191726,6378c7e85a20c5676abc7421,2022-11-27 21:21:34,2043.433173,Dr. Buhmann Akademie H


In [51]:
x = [current_ratings[item]['rating'] for item in current_ratings.keys()]
fig = go.Figure(data=[go.Histogram(x=x)])
fig.update_layout(title='Naive Elo Distribution')
fig

In [52]:
teams2plot = {
    'Spacestation Gaming', 'NRG Esports', 'Version1','FaZe Clan', 'FURIA Esports', 'Shopify Rebellion', 'Team AXLE', 'Gen.G Esports', 'G2 Esports'
}
plot_df = ratings_history[(ratings_history['name'].apply(lambda x: x in teams2plot)) & (ratings_history['date'].dt.year >= 2021)].copy()
plot_df['month'] = plot_df['date'].to_numpy().astype('datetime64[M]')
plot_df = plot_df.groupby(['month', 'name'], as_index=False).mean(['rating'])
px.line(plot_df, x='month', y='rating', color='name')

# Player core stats

In [12]:
# Extract data based on player id
player_id = '5f3d8fdd95f40596eae23dcf'
with open('queries/player_core_stats.sql') as f:
    sql = f.read()
sql = sql.format(where=f"where gp.id = '{player_id}'")
df = pd.read_sql(sql, cnx)

## Plot data
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
for item in set(df.columns) - {'month'}:
    if item in {'avg_game_score', 'monthly_games'}:
        fig.add_trace(go.Scatter(x=df['month'], y=df[item], mode='lines', name=item), secondary_y=True)
    else:
        fig.add_trace(go.Scatter(x=df['month'], y=df[item], mode='lines', name=item))

# Add figure title
player_name = cnx.execute(f"select tag from rocket_league.players where id='{player_id}'").fetchall()[0][0]
fig.update_layout(title_text=f'Core stats for {player_name}')


fig